# Linear Regression using Ordinary Least Squares

In [1]:
import os
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import train_test_split

In [2]:
path = r'E:\1MS\Projects\LinearRegression_using_OLS'
os.chdir(path)

In [3]:
def read_approximation_files(fname, N, M):
    """ This function reads files """
    try:
        with open(fname) as f:
            data = []
            for line in f:
                data.extend(line.split())
                
    except IOError:
        print("Cannot open file")
        sys.exit(1)
        
    data = [float(i) for i in data]
    data = np.reshape(data, (len(data)//(N+M), (N+M)))
    x = data[:, :N]
    t = data[:, N:]
    Nv = len(x)
    
    return x, t, Nv

    
class Standardize():
    """ This class scales the data to have zero mean and unit variance """
    
    def fit(self, x):
        self.mean = x.mean(axis=0)
        self.std = x.std(axis=0)
        return 
        
    
    def transform(self, x):
        x_t = x - self.mean
        x_t = x_t / self.std
        return x_t
        
        
    def fit_transform(self, x):
        self.fit(x)
        x_t = self.transform(x)
        return x_t


def add_constant(x):
    """ This functions adds the intercept/constant """
    
    n = np.shape(x)[0]
    xa = np.concatenate( (np.ones((n,1), dtype=float) , x),1)
    return xa
    
    
def get_preds(x, coef):
    """ Returns the predicted values calculated from coefficients/ weights """
    
    return np.dot(x, coef) 


def calc_error(y, y_pred, metric='mse'):
    """ 
        Calculates the error between actual and predicted values.
        Uses mean squared error metric by default. 
        By passing metric='sse', sum of squared error metric can be used
    """
    
    Nv_ = np.shape(y)[0]
    loss = y - y_pred
    
    if metric == 'sse':
        feature_error = sum(loss**2)/Nv_
        try:
            error = sum(feature_error)
        except TypeError: # if there's only one dependent variable 
            error = feature_error
    
    elif metric == 'mse':
        error = np.mean(loss**2)

    return error


#Helper functions    
def get_source(lib):
    ''' This function prints the source code of required library '''
    
    import inspect
    print(inspect.getsource(lib))
    
    
def gs(x, r=False):
    """ gs: get shape - This function prints or returns the shape of given array """
    
    if r:
        return np.shape(x)
    print(np.shape(x))
    
    
def df(x):
    """ df- returns input as a dataframe """
    
    return pd.DataFrame(x)


## Training

In [4]:
fname = 'Twod.tra'
N, M = 8, 7

In [5]:
x, y, Nv = read_approximation_files(fname, N, M)

In [6]:
gs(x)
gs(y)

(1768, 8)
(1768, 7)


Splitting the data into training and validation

In [7]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.25, random_state=3, shuffle=True)

In [8]:
gs(x_train)
gs(y_train)
gs(x_val)
gs(y_val)

(1326, 8)
(1326, 7)
(442, 8)
(442, 7)


In [9]:
df(x_train).describe()

,0,1,2,3,4,5,6,7
count,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000
mean,-15.689281,-15.857155,-19.715648,-20.965447,-22.497912,-25.395404,-25.437874,-31.304241
std,3.450740,3.443045,2.648086,2.592289,2.584743,2.587408,2.954021,2.883025
min,-25.552600,-25.740200,-27.581900,-29.015100,-32.430900,-36.430800,-36.580900,-43.926500
25%,-18.141025,-18.304900,-21.471450,-22.534475,-24.028050,-26.892475,-27.196575,-32.855050
50%,-15.893100,-16.066350,-19.832300,-21.059650,-22.391650,-24.996050,-25.389650,-30.802000
75%,-13.269175,-13.451600,-17.958750,-19.266625,-20.890000,-23.749375,-23.725775,-29.527925
max,-6.571330,-6.752560,-12.558700,-13.987600,-15.527300,-19.037200,-17.206300,-24.222900


Scaling the data

In [10]:
train_scale = Standardize()
x_train_scaled = train_scale.fit_transform(x_train)

In [11]:
df(x_train_scaled).describe()

,0,1,2,3,4,5,6,7
count,1.326000e+03,1.326000e+03,1.326000e+03,1.326000e+03,1.326000e+03,1.326000e+03,1.326000e+03,1.326000e+03
mean,-1.035002e-14,-1.160124e-15,4.947274e-15,-9.380129e-15,1.081019e-14,6.446996e-15,2.151455e-15,-3.248616e-15
std,1.000377e+00,1.000377e+00,1.000377e+00,1.000377e+00,1.000377e+00,1.000377e+00,1.000377e+00,1.000377e+00
min,-2.859399e+00,-2.871520e+00,-2.971664e+00,-3.106401e+00,-3.844380e+00,-4.266647e+00,-3.773579e+00,-4.379782e+00
25%,-7.107663e-01,-7.111927e-01,-6.632960e-01,-6.054958e-01,-5.922116e-01,-5.788168e-01,-5.955831e-01,-5.381132e-01
50%,-5.908771e-02,-6.078163e-02,-4.406811e-02,-3.635347e-02,4.112692e-02,1.544036e-01,1.633087e-02,1.742722e-01
75%,7.015941e-01,6.989344e-01,6.637100e-01,6.555838e-01,6.223129e-01,6.364092e-01,5.798010e-01,6.163619e-01
max,2.643315e+00,2.645342e+00,2.703707e+00,2.692786e+00,2.697847e+00,2.458291e+00,2.787617e+00,2.457146e+00


We can observe, the training data is now zero mean and of unit variance

In [12]:
x_val_scaled = train_scale.transform(x_val)

In [13]:
x_train_ = add_constant(x_train_scaled)
x_val_ = add_constant(x_val_scaled)

In [14]:
gs(x_train_)
gs(x_val_)

(1326, 9)
(442, 9)


---
# 1. OLS - directly solving Normal Equations

In [15]:
def ne_OLS(x,y):
    """ Returns coefficients by solving normal equations directly. """
    
    Nv = np.shape(x)[0]
    R = np.dot(x.T, x)/Nv
    C = np.dot(x.T, y)/Nv

    if R.ndim < 2:
        coef = C/R
    else:
        coef = np.dot(np.linalg.inv(R), C)
        
    return coef


In [16]:
coef_ne = ne_OLS(x_train_, y_train)
gs(coef_ne)

(9, 7)


In [17]:
preds_ne = get_preds(x_val_, coef_ne)
gs(preds_ne)

(442, 7)


In [18]:
error_ne = calc_error(y_val, preds_ne, metric='mse')
error_ne

0.04806708112056249

#### Math intuition
- Our aim is to find the best possible coefficients (coef/ weights), from the equation: $ X \cdot coef = y $
- i.e to minimize: $ || X \cdot coef - y ||^2 $
- On further simplication, we get the Normal equations, given as: $ (X^T \cdot X) \cdot coef = X^T \cdot y $
- Thus, here we find the coefficients (or weights) as: $$ coef = (X^T \cdot X)^{-1} \cdot (X^T \cdot y) $$

---
# 2. OLS using QR decomposition

In [19]:
def qr_OLS(x,y):
    """ This function uses QR decomposition to solve the normal equations and returns the coefficients. """
    
    Nv = np.shape(x)[0]
    Q, R = np.linalg.qr(x) #factorization of x into Q and R
    C = np.dot(Q.T, y)
    
    coef = np.dot(np.linalg.inv(R), C)
        
    return coef, Q, R


In [20]:
coef_qr, q, r = qr_OLS(x_train_, y_train)
gs(coef_qr)

(9, 7)


In [21]:
preds_qr = get_preds(x_val_, coef_qr)
gs(preds_qr)

(442, 7)


In [22]:
error_qr = calc_error(y_val, preds_qr, metric='mse')
error_qr

0.04806708112046354

#### Math intuition
- QR(X) decomposes input X into Q - orthogonal matrix and R - Upper triangular matrix.
- QR decomposition method is more stable because it avoids forming: $ X^T \cdot X $
- Substituting X as QR in the normal equations, we get the solution to be: $ R \cdot coef = Q^T \cdot y $
- Inverse of orthogonal matrix Q is nothing but it's transpose, thus, no complex computation is required. Inverting an upper triangular matrix is faster than inverting $ X^T \cdot X $

---
# 3. OLS using SVD and PseudoInverse

In [23]:
def svd_OLS(x,y):
    """ This function uses SVD decomposition and pseudoinverse to solve the normal equations and returns the coefficients."""
    
    Nv = np.shape(x)[0]
    u, s, vt = scipy.linalg.svd(x, full_matrices=False)
    
    s_nz = s[s>0] #Selecting the non-zero singular values 
    s_inv = np.array([1/i for i in s_nz]) # Inverting the non-zero singular values. 
    
    m, n = len(s), len(s_inv)
    if m != n:
        s_inv = np.append(s_inv, np.zeros((m-n,1),float))
        
    s_inv_ = np.diag(s_inv) # converting 1D array to 2-D diagonal matrix

    pseudo_inv = vt.T @ s_inv_ @ u.T 

    coef = np.dot(pseudo_inv, y)
    
    # One can directly use scipy or numpy's linalg.pinv(x) 
    # coef = np.dot( np.linalg.pinv(x), y)
    
    return coef

In [24]:
coef_svd = svd_OLS(x_train_, y_train)
gs(coef_svd)

(9, 7)


In [25]:
preds_svd = get_preds(x_val_, coef_svd)
gs(preds_svd)

(442, 7)


In [26]:
error_svd = calc_error(y_val, preds_svd, metric='mse')
error_svd

0.04806708112046273

#### Math intuition
- SVD numerically accurate and stable method, but at the same time it is a bit computationally expensive than other methods.
- We decompose X into orthogonal matrices U and V transpose, and a diagonal matrix, S: $ X = U \cdot S \cdot V^T $
- Substituting X as above in the Normal equations and further simplifying it,  gives us: 
$$ coef = (V \cdot S^{-1} \cdot U^T) \cdot y  $$
- Now, if any of the singular values of S equals 0 then S inverse doesn't exist. In such case, we perform pseudoinverse. i.e. we take reciprocal of non-zero singular values and leave all zero singular values at zero.
- This new matrix is called as the pseudoinverse of S, denoted as:  $ S^+ $
- And hence, in such cases we get pseudoinverse of X, as:  $ X^+ = V \cdot S^+ \cdot U^T $
- Finally we can compute the weights/ coefficients as: $$ coef = X^+ \cdot y $$


---
# Comparing with sklearn's LinearRegression

In [27]:
from sklearn.linear_model import LinearRegression

In [28]:
lr = LinearRegression(fit_intercept=False)
lr.fit(x_train_, y_train)

LinearRegression(fit_intercept=False)

In [29]:
preds_lr = lr.predict(x_val_)

In [30]:
error_lr = calc_error(y_val, preds_lr, metric='mse')
error_lr

0.048067081120463366

### Conclusion
- We can see, all three methods gave approximately similar results as from Linear Regression.
- sklearn's LR uses SVD (LAPACK's SVD), with Divide and Conquer approach.
- Though, for the given data we got similar results, some of the data may not work well with the 1st approach of directly inverting $ X^T \cdot X $
- This notebook demonstrates some of the commonly used methods used for solving linear least squares problems.

---
## Testing

- Fitting the complete data (x) and testing on actual test data

In [31]:
fname_test = 'Twod.tst'
N, M = 8, 7

In [32]:
x_test, y_test, Nv_test = read_approximation_files(fname_test, N, M)

In [33]:
gs(x_test)
gs(y_test)

(1000, 8)
(1000, 7)


#### Processing data like earlier

In [34]:
test_scale = Standardize()
x_scaled = test_scale.fit_transform(x) # x is the complete training data loaded at the beginning of this notebook.

In [35]:
gs(x)
gs(y)

(1768, 8)
(1768, 7)


In [36]:
x_test_scaled = test_scale.transform(x_test)

In [37]:
x_ = add_constant(x_scaled)
x_test_ = add_constant(x_test_scaled)

#### OLS - SVD

In [38]:
coef_svd_all = svd_OLS(x_, y)
gs(coef_svd_all)

(9, 7)


In [39]:
preds_svd_test = get_preds(x_test_, coef_svd_all)
gs(preds_svd_test)

(1000, 7)


In [40]:
error_svd_test = calc_error(y_test, preds_svd_test, metric='mse')
error_svd_test

0.04987707320312501

#### sklearn's Linear Regression

In [41]:
lr_x = LinearRegression(fit_intercept=False)
lr_x.fit(x_, y)

LinearRegression(fit_intercept=False)

In [42]:
preds_lr_test = lr_x.predict(x_test_)

In [43]:
error_lr_test = calc_error(y_test, preds_lr_test, metric='mse')
error_lr_test

0.04987707320312508

Hence, we obtained approximately equal test results using OLS-SVD and sklearn's linear regression on completely unknown test data.